<a href="https://colab.research.google.com/github/GuilhermeRuy97/Data-Science-em-Producao-Meigarom/blob/main/data_science_em_producao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modulo 02 Video 01
-Store Sales Prediction


## 0.0 IMPORTS

In [20]:
#!pip install inflection

In [32]:
import pandas as pd
import inflection
import math

### 0.1 Helper Functions

### 0.2 Loading Data

In [22]:
df_sales_raw = pd.read_csv( 'train.csv', low_memory = False)
df_store_raw = pd.read_csv( 'store.csv', low_memory = False)

#Fazendo um merge dos arquivos guardando em "store"

df_raw = pd.merge( df_sales_raw, df_store_raw, how = 'left', on = 'Store' )

In [23]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
642132,678,2,2013-12-03,9328,1019,1,1,0,0,c,a,3250.0,NaN,NaN,1,40.0,2011.0,"Jan,Apr,Jul,Oct"


## 1.0 DESCRICAO DOS DADOS

In [24]:
#É interessante fazer uma cópia dos dados ao mudar de sessão para resgatá-los no futuro.

df1 = df_raw.copy()

### 1.1 Rename Columns

In [25]:
#Nem sempre os dados vem com nomes adequados para o desenvolvimento dos algoritmos;
#Neste caso é mais controlado pois são dados do kaggle.

df_raw.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [26]:
#Transformando os dados de camel case para snake case

#Colunas originais

cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
            'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
            'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
            'Promo2SinceYear', 'PromoInterval']

#Mudando coluna para snake case

snakecase = lambda x: inflection.underscore( x )

cols_new = list( map( snakecase, cols_old ) )

#Rename

df1.columns = cols_new

In [27]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

### 1.2 Data Dimensions

In [28]:
print( f'Number of Rows: { df1.shape[0] }')
print( f'Number of Columns: { df1.shape[1] }')


Number of Rows: 1017209
Number of Columns: 18


### 1.3 Data Types

In [29]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [30]:
#Transformando a data de object para data

df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

### 1.4 Check NA

In [31]:
#Mostrando a soma de todas as linhas que tenha pelo menos 1 na

df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

### 1.5 Fillout NA

In [ ]:
df1['competition_distance'.max]

In [44]:
# competition_distance
# Trocando o valor em NA pelo maior valor:
df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000.0 if math.isnan(x) else x)

# competition_open_since_month    
# Trocando a data de "competição desde o mes" pela data de venda da ultima compra
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month'] ) else x['competition_open_since_month'], axis = 1)

# competition_open_since_year     
# Mesmo processo do mes para o ano
df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['competition_open_since_year'] ) else x['competition_open_since_year'], axis = 1)

# promo2                               
# promo2_since_week               
df1['promo2_since_week'] = df1.apply( lambda x: x['date'].week if math.isnan( x['promo2_since_week'] ) else x['promo2_since_week'], axis = 1)

# promo2_since_year               
df1['promo2_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['promo2_since_year'] ) else x['promo2_since_year'], axis = 1)

# promo_interval                  
month_map = {
1: 'Jan', 
2: 'Feb', 
3: 'Mar', 
4: 'Apr', 
5: 'May', 
6: 'Jun', 
7: 'Jul', 
8: 'Ago', 
9: 'Sep', 
10: 'Oct', 
11: 'Nov', 
12: 'Dec' }

df1['promo_interval'].fillna(0, inplace = True)

df1['month_map'] = df1['date'].dt.month.map( month_map )

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis = 1 )

In [45]:
#Repare que a soma agora de NA's é 0

df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

## 1.6

## 1.7

## 1.8